# Libraries

Import your libraries

In [2]:
import numpy as np
import pandas as pd
import statistics as cal
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, plot_confusion_matrix, confusion_matrix

# Importing Data

Import the CSV file with Actions, Sum and Div as a Dataframe called df. Fill the empty values of Action with 0. 
Replace NaN values with 0. Delete first 100 rows. 

In [4]:
#load in df
Player = 9 
Game = 2

df = pd.read_csv('matrix_Player_' + str(Player) + '_game_' + str(Game) + '_All_Action.csv')

# Fill NaN with 0
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.fillna(0)

#Convert Sprinting to 1
df.Action[df.Action == "Sprinting"] = 1

#Delete first 99 rows, so df starts at timeLine == 1.00
df = df.iloc[99: , :]

df_All = df

<PandasArray>
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
 ...
 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Length: 754126, dtype: float64


<ipython-input-4-0ffc0f660cd8>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Action[df.Action == "Sprinting"] = 1


# Preparing Data

Split the Dataframe in chunks van n/100 seconden. Combine to 1 dataframe. With Sum,Div and Action split into Train and Valid/Test. 

In [159]:
#Convert data into chunks of a second
n = 50 #chunk size (100 datapoints in one second)
chunks = [df[i:i+n] for i in range(0,df.shape[0],n)]

#Set chunks into dataframe
#for chunk in chunks:
frames = []
for chunk in chunks:
    P = pd.DataFrame({'Frames':[chunk['Unnamed: 0'].tolist()], 'frAcc':[max(chunk['frAcc'].tolist())], 
                      'frRoAcc':[chunk['frRoAcc'].tolist()], 'frDispl':[chunk['frDispl'].tolist()], 
                      'frRoAng':[chunk['frRoAng'].tolist()], 'frSpeed':[max(chunk['frSpeed'].tolist())], 
                      'timeLine':[max(chunk['timeLine'].tolist())], 'frameRotationalSpeedX':[chunk['frameRotationalSpeedX'].tolist()], 
                      'frameRotationalSpeedY':[chunk['frameRotationalSpeedY'].tolist()], 'frameRotationalSpeedZ':[(chunk['frameRotationalSpeedZ'].to_numpy())],
                      'wheelRotationalSpeedX':[chunk['wheelRotationalSpeedX'].to_numpy()], 'wheelRotationalSpeedY':[chunk['wheelRotationalSpeedY'].tolist()],
                      'wheelRotationalSpeedZ':[chunk['wheelRotationalSpeedZ'].to_numpy()], 'frRoSpeed':[max(chunk['frRoSpeed'].tolist())],
                      'Sum_WheelX_FrameZ':[max(chunk['Sum_WheelX_FrameZ'].tolist())], 'Div_FrameZ_WheelX':[cal.mean(chunk['Div_FrameZ_WheelX'].tolist())],
                      'Filt_WheelX':[cal.mean(chunk['Filt_WheelX'].tolist())], 'Filt_FrameZ':[cal.mean(chunk['Filt_FrameZ'].tolist())],
                      'Action':[chunk['Action'].to_numpy()]})
    frames.append(P)

df = pd.concat(frames,ignore_index=True)

In [164]:
# split chunk data into train test validate (with colum [sum and Div] as input, and action as output)
#X_train, X_test, y_train, y_test = train_test_split(df[[
#                                                        'Sum_WheelX_FrameZ',
#                                                        'Div_FrameZ_WheelX',
#                                                        'wheelRotationalSpeedX',
#                                                        'Filt_FrameZ',
#                                                        'Filt_WheelX',
#                                                       ]], 
#                                                    df.Action, test_size = 0.2, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(df[['wheelRotationalSpeedX', 'frameRotationalSpeedZ']], 
                                                    df[['Action']], test_size = 0.2, random_state=42)

X_train = X_train[['wheelRotationalSpeedX','frameRotationalSpeedZ']].values
y_train = y_train['Action'].values
y_train


<ipython-input-164-a73351cb3fff>:17: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  f = np.vectorize(np.int)


# Fitting Model

Choose a classification model. Train model with Training data.

In [167]:
model2 = DecisionTreeClassifier(random_state=6)
model2.fit(X_train,y_train)

ValueError: setting an array element with a sequence.

# Validate/Tune Model

Validate results of the model (Precision/Recall). Tune the parameters of the model to achieve better results

In [ ]:
#GridsearchCV Polynomials
plot_confusion_matrix(model2, X_test, y_test)
plt.show()

# Export Data

Export the results in CSV format. Layout = Time,Action